## Import and Load data file


nltk: Natural Language Toolkit for text processing tasks like tokenization and lemmatization.
WordNetLemmatizer: Part of NLTK used for lemmatizing words.
json: Library for handling JSON data.
pickle: Python serialization library for saving Python objects.
numpy: Library for numerical operations.
Sequential, Dense, Activation, Dropout: Components from Keras for building a sequential neural network.
SGD: Stochastic Gradient Descent optimizer from Keras.

**Variables:**
words: List to store unique words from the intents.
classes: List to store unique classes (intent tags).
documents: List to store tuples of tokenized words and their corresponding intent tags.
ignore_words: List of punctuation marks to ignore during processing.
Data Loading:
Reads the content of the intents.json file, which presumably contains intents with associated patterns and tags.
Processing Intents:
Iterates over each intent in the loaded JSON data.
Tokenizes each pattern within the intent, lemmatizes the words, and adds them to the words list.
Builds a list of tuples (documents) containing tokenized words and their associated intent tags.
Populates the classes list with unique intent tags.

**Preprocessing:**
Lemmatizes and lowercases words in the words list, removing duplicates and sorting them alphabetically.
Sorts the classes list alphabetically.
Data Preparation:
Converts patterns and intents into a format suitable for training a neural network.
Creates a bag-of-words representation for each pattern and generates corresponding output vectors.

**Neural Network Model:**
Builds a sequential neural network using Keras.
Defines the architecture with multiple dense layers, activation functions (ReLU and Softmax), and dropout layers.
Compiles the model using a Stochastic Gradient Descent (SGD) optimizer and categorical cross-entropy loss.

**Model Training:**
Fits the model using the prepared training data (train_x and train_y).

**Model Saving:**
Saves the trained model (chatbot_model.h5) and associated training history.

**Model Loading:**
Loads the trained model (chatbot_model.h5) for making predictions.

**Utility Functions:**
Defines utility functions like clean_up_sentence, bow, predict_class, and chatbot_response for text preprocessing and making predictions.

In [2]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

## Pre-process data

**Looping Through Intents and Patterns:**

The code iterates over each intent in the intents dictionary.
For each intent, it further iterates over its associated patterns.

**Tokenization and Document Preparation:**
Each pattern (sentence) within an intent is tokenized into individual words using NLTK's word_tokenize function.
The tokenized words (w) are then extended into the words list.
A tuple (w, intent['tag']) is appended to the documents list, which represents a document (pattern) along with its associated intent tag.

**Building Vocabulary and Intent Classes:**
Checks if the intent tag (intent['tag']) is not already in the classes list. If not, it adds the tag to classes.
Lemmatizes each word (using WordNetLemmatizer) and converts it to lowercase, excluding words listed in ignore_words.
Removes duplicates from the words list, sorts them alphabetically, and assigns the result back to words.
Sorts the classes list alphabetically for consistent ordering.

**Output:**
Prints the number of documents (combinations of patterns and intents) processed.
Prints the total number of unique intent classes (classes) and unique lemmatized words (words).

**Data Serialization:**
Serializes (saves) the words and classes lists into binary files (words.pkl and classes.pkl) using pickle.
This allows the vocabulary (words) and intent classes (classes) to be reused for model training and inference.

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [43]:
# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

105 documents
22 classes ['cancel', 'confirm', 'delay', 'delivery', 'goodbye', 'greeting', 'items', 'mobile app', 'more', 'order status', 'payments', 'personal', 'profile info', 'reasons', 'refund', 'refund status', 'return', 'thanks', 'track', 'use voucher', 'user response', 'withdraw refunds']
145 unique lemmatized words ["'m", "'s", '.', 'a', 'about', 'accept', 'account', 'address', 'agent', 'am', 'an', 'any', 'anyone', 'app', 'are', 'available', 'be', 'become', 'but', 'buy', 'bye', 'call', 'can', 'cancel', 'cancelled', 'card', 'cash', 'change', 'common', 'completed', 'confirmation', 'confirmed', 'contact', 'credit', 'credited', 'customer', 'day', 'deducted', 'delay', 'delayed', 'delete', 'delivered', 'delivery', 'did', 'do', 'doe', 'doing', 'download', 'even', 'for', 'from', 'fund', 'get', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helpline', 'hey', 'hi', 'how', 'i', 'im', 'information', 'is', 'it', 'item', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'm

## Create training and testing data

**Initialization:**

Creates an empty list training to store processed training data.
Initializes output_empty as a list of zeros with a length equal to the number of unique intent classes (classes).

**Document Processing:**

Iterates through each doc in the documents list.
For each document (doc), initializes an empty list bag to represent the bag-of-words vector for the pattern.
Lemmatizes and converts each word in the pattern (doc[0]) to lowercase using WordNetLemmatizer.
Constructs the bag-of-words vector (bag) for the current pattern by checking the presence of each word (w) from the vocabulary (words) and appending 1 if the word is present, otherwise appends 0.

**Intent Labeling:**

Creates an output_row list initialized with zeros (output_empty) to represent the intent label vector for the current document (doc).
Sets the appropriate index of output_row to 1 based on the position of the intent class (doc[1]) in the classes list. This step encodes the intent label in a one-hot encoded format.

**Training Data Creation:**
Appends the tuple (bag, output_row) to the training list, where bag is the bag-of-words vector and output_row is the corresponding intent label vector.

**Shuffling and Array Conversion:**
Randomly shuffles the training data to ensure that the model does not learn based on any particular order of patterns.
Extracts the bag-of-words vectors (train_x) and intent label vectors (train_y) from the training list and converts them into NumPy arrays for compatibility with machine learning models.

In [4]:
import numpy as np
import random

training = []
output_empty = [0] * len(classes)

# Iterate through documents and process each pattern
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # Create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # Create output row (intent) array
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    # Append the bag of words and output row as a tuple to training
    training.append((bag, output_row))

# Shuffle the training data
random.shuffle(training)

# Extract bag and output_row arrays from the training data
train_x = np.array([x[0] for x in training])
train_y = np.array([x[1] for x in training])

print("Training data created")


Training data created


## Create Model

**Model Definition:**

Initializes a sequential model model using Sequential() from Keras.
Adds a dense layer (Dense) with 128 units and a rectified linear unit (ReLU) activation function. The input_shape is specified as the length of the bag-of-words vector (len(train_x[0])).
Adds a dropout layer (Dropout) with a dropout rate of 50% (0.5) to prevent overfitting.
Adds another dense layer with 64 units and ReLU activation, followed by another dropout layer.

**Output Layer:**

Adds a dense output layer with the number of units equal to the length of the intent label vector (len(train_y[0])) and uses the softmax activation function to obtain probability scores across all intent classes.

**Model Compilation:**

Configures the model for training by specifying the loss function (categorical_crossentropy), optimizer (SGD with Nesterov momentum), and evaluation metric (accuracy).

**Model Training:**
Fits the model (model.fit) using the training data (np.array(train_x) and np.array(train_y)) for a specified number of epochs (200) and batch size (5). The training progress is displayed (verbose=1).

**Model Saving:**
Saves the trained model to a file named 'chatbot_model.h5' along with training history (hist).

In [7]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
21/21 [==============================] - 0s 3ms/step - loss: 3.2058 - accuracy: 0.0476
Epoch 2/200
21/21 [==============================] - 0s 3ms/step - loss: 2.9252 - accuracy: 0.1619
Epoch 3/200
21/21 [==============================] - 0s 3ms/step - loss: 2.6127 - accuracy: 0.2667
Epoch 4/200
21/21 [==============================] - 0s 2ms/step - loss: 2.3945 - accuracy: 0.3143
Epoch 5/200
21/21 [==============================] - 0s 2ms/step - loss: 2.3378 - accuracy: 0.3333
Epoch 6/200
21/21 [==============================] - 0s 2ms/step - loss: 2.1912 - accuracy: 0.3905
Epoch 7/200
21/21 [==============================] - 0s 2ms/step - loss: 1.9025 - accuracy: 0.4476
Epoch 8/200
21/21 [==============================] - 0s 2ms/step - loss: 1.8462 - accuracy: 0.4381
Epoch 9/200
21/21 [==============================] - 0s 3ms/step - loss: 1.6622 - accuracy: 0.4952
Epoch 10/200
21/21 [==============================] - 0s 2ms/step - loss: 1.6197 - accuracy: 0.5048
Epoch 11/

21/21 [==============================] - 0s 3ms/step - loss: 0.6871 - accuracy: 0.7524
Epoch 84/200
21/21 [==============================] - 0s 3ms/step - loss: 0.8376 - accuracy: 0.7810
Epoch 85/200
21/21 [==============================] - 0s 3ms/step - loss: 0.6070 - accuracy: 0.7905
Epoch 86/200
21/21 [==============================] - 0s 2ms/step - loss: 0.7119 - accuracy: 0.7619
Epoch 87/200
21/21 [==============================] - 0s 2ms/step - loss: 0.7308 - accuracy: 0.7810
Epoch 88/200
21/21 [==============================] - 0s 3ms/step - loss: 0.7740 - accuracy: 0.7524
Epoch 89/200
21/21 [==============================] - 0s 3ms/step - loss: 0.6044 - accuracy: 0.7714
Epoch 90/200
21/21 [==============================] - 0s 3ms/step - loss: 0.6521 - accuracy: 0.8000
Epoch 91/200
21/21 [==============================] - 0s 3ms/step - loss: 0.7700 - accuracy: 0.7524
Epoch 92/200
21/21 [==============================] - 0s 3ms/step - loss: 0.7081 - accuracy: 0.7619
Epoch 93/200


21/21 [==============================] - 0s 3ms/step - loss: 0.9290 - accuracy: 0.7619
Epoch 165/200
21/21 [==============================] - 0s 3ms/step - loss: 0.5040 - accuracy: 0.8190
Epoch 166/200
21/21 [==============================] - 0s 3ms/step - loss: 0.4968 - accuracy: 0.8286
Epoch 167/200
21/21 [==============================] - 0s 2ms/step - loss: 0.4910 - accuracy: 0.8571
Epoch 168/200
21/21 [==============================] - 0s 2ms/step - loss: 0.5642 - accuracy: 0.8095
Epoch 169/200
21/21 [==============================] - 0s 3ms/step - loss: 0.5167 - accuracy: 0.8286
Epoch 170/200
21/21 [==============================] - 0s 2ms/step - loss: 0.4957 - accuracy: 0.8381
Epoch 171/200
21/21 [==============================] - 0s 2ms/step - loss: 0.5682 - accuracy: 0.8000
Epoch 172/200
21/21 [==============================] - 0s 3ms/step - loss: 0.4840 - accuracy: 0.7905
Epoch 173/200
21/21 [==============================] - 0s 3ms/step - loss: 0.5325 - accuracy: 0.8286
Epoc

**Model Loading:**
Loads the trained chatbot model (chatbot_model.h5) along with vocabulary and intent data required for intent recognition.

**Text Processing:**
Provides functions (clean_up_sentence and bow) to preprocess user input for intent prediction.

**Intent Prediction:**
Utilizes the loaded model to predict the intent of user queries and returns the predicted intent along with associated probabilities.

**Response Generation:**
Generates appropriate responses based on predicted intents, ensuring dynamic and context-aware interaction with users.

In [8]:
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [9]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

In [10]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

In [11]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [12]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

In [13]:
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

## GUI Interface

**GUI Layout:**

 **Window Settings:**
 Sets up a Tkinter window titled "E-Commerce Chatbot" with specific dimensions (width=800, height=800), background color  (bg=BG_COLOR), and other configurations.
 
 **Chat Window:**
Creates a Text widget (ChatLog) to display chat messages. Configures it with specific fonts, colors, and disables user editing (state=DISABLED). Uses a Scrollbar to scroll through messages.
 
 **Head Label:**
Displays a header label (head_label) at the top of the window with the text "eBuddy", styled with specific fonts and colors.

 **Separator Line:**
Creates a horizontal line (line) below the header label to separate it from the chat window.

 **Input Box & Send Button:**
Sets up an input Text widget (EntryBox) for users to type messages. Binds the <Return> key to the send function for sending messages.
Creates a Button (SendButton) labeled "Send" to submit messages. Configures its appearance with specific font, colors, and a command to call the send function.

    **Event Handling (send Function):

    **Send Function (send):
Defines the send function to handle user input events.
Retrieves the message from EntryBox, processes it (e.g., removes trailing whitespace), and clears the input box.
Displays the user's message in the chat window (ChatLog) as "You: <message>".
Calls the chatbot_response function to get a response for the user's message.
Displays the chatbot's response in the chat window as "Bot: <response>".

    **Integration with Chatbot Logic:
The send function integrates with a chatbot logic (assumed to be defined elsewhere, e.g., chatbot_response function) to simulate a conversation between the user and the chatbot.

    **Overall Flow:
Users interact by typing messages in EntryBox and clicking the "Send" button.
Sent messages and chatbot responses are displayed in ChatLog

In [59]:
import tkinter
from tkinter import *

BG_GRAY = "#0a192f"
BG_COLOR = "#f0f0f0"
TEXT_COLOR = "#000000"


FONT = "Helvetica 14"
FONT_BOLD = "Helvetica 13 bold"


def send(event):
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#000000", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Bot: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
        
        
base = Tk()
base.title("E-Commerce Chatbot")
base.resizable(width=FALSE, height=FALSE)
base.configure(width=800, height=800, bg=BG_COLOR)


#Create Chat window
ChatLog = Text(base, bd=0, bg=BG_COLOR, fg=TEXT_COLOR, font=FONT_BOLD)
ChatLog.config(state=DISABLED)

head_label = Label(base, bg=BG_COLOR, fg=TEXT_COLOR, text="eBuddy", font=FONT_BOLD, pady=10)
head_label.place(relwidth=1)

line = Label(base, width=450, bg=BG_GRAY)


#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set
ChatLog.focus()

#Create Button to send message
SendButton = Button(base, font=("Verdana", 12,'bold'), text="Send", width="12", height=15,
                    bd=0, bg="#ed9061", activebackground="#3c9d9b",fg='#ffffff',
                    command=lambda: send)

#Create the box to enter message
EntryBox = Text(base, bg="white",width="29", height="5", font="Arial", background="#dddddd")
EntryBox.focus()
EntryBox.bind("<Return>", send)
#EntryBox.bind("<Return>", send)



scrollbar.place(x=775,y=6, height=800)
line.place(x=0,y=35, height=1, width=770)
ChatLog.place(x=5,y=40, height=700, width=770)
EntryBox.place(x=0, y=740, height=60, width=600)
SendButton.place(x=600, y=740, height=60, width=175)



In [60]:
base.mainloop()

1/1 [==============================] - 0s 26ms/step
